In [1]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('eventresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [5]:
import logging

class EventSpider(scrapy.Spider):
    name = 'eventbrite_events'
    
    file = open('cityresult.json')
    data = json.load(file)
    file.close()
    
    start_urls = []
    
    for index in range(len(data)):
        
        for (k,v) in data[index].items():
            
            city_tag = str(v)
            city_tag = city_tag[:-7]
            city_url = f'https://www.eventbrite.com{city_tag}conference-events/'
            
            start_urls.append(city_url)

    print(start_urls)
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'eventresult.json'                        # Used for pipeline 2
    }

    def parse(self, response):
        for event in response.css('main.eds-media-card-content__content-container'):
            yield {
                'name': event.css('div.eds-is-hidden-accessible::text').get(),
                'date': event.css('div.eds-media-card-content__sub-content > div.eds-text-bs--fixed::text').get(),
                'location': event.css('div.eds-text-bs--fixed > div.card-text--truncated__one::text').get(),
                'ticket-price': event.css('div.eds-media-card-content__sub-content-cropped > div.eds-text-bs--fixed::text').get(),
                'organizer-link': event.css('div.eds-media-card-content__primary-content > a::attr("href")').get(),
            }

        next_page_url = response.css("div.eds-l-pad-left-4 > a.eds-btn::attr(href)").extract_first()
        if next_page_url is not None:
            yield scrapy.Request(response.urljoin(next_page_url))

['https://www.eventbrite.com/d/al--alabaster/conference-events/', 'https://www.eventbrite.com/d/al--anchorage/conference-events/', 'https://www.eventbrite.com/d/al--anniston/conference-events/', 'https://www.eventbrite.com/d/al--athens/conference-events/', 'https://www.eventbrite.com/d/al--auburn/conference-events/', 'https://www.eventbrite.com/d/al--austin/conference-events/', 'https://www.eventbrite.com/d/al--bay-minette/conference-events/', 'https://www.eventbrite.com/d/al--bessemer/conference-events/', 'https://www.eventbrite.com/d/al--birmingham/conference-events/', 'https://www.eventbrite.com/d/al--blountsville/conference-events/', 'https://www.eventbrite.com/d/al--boaz/conference-events/', 'https://www.eventbrite.com/d/al--calera/conference-events/', 'https://www.eventbrite.com/d/al--carrollton/conference-events/', 'https://www.eventbrite.com/d/al--cullman/conference-events/', 'https://www.eventbrite.com/d/al--daphne/conference-events/', 'https://www.eventbrite.com/d/al--decatur

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(EventSpider)
process.start()

2019-08-04 16:20:06 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-08-04 16:20:06 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.1, Python 3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.4.2, Platform Windows-10-10.0.17134-SP0
2019-08-04 16:20:06 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'eventresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-08-04 16:33:58 [scrapy.core.scraper] ERROR: Error downloading <GET https://www.eventbrite.com/d/argentina--san-nicolas/conference-events/>
Traceback (most recent call last):
  File "C:\Users\bella\Miniconda3\lib\site-packages\scrapy\core\downloader\middleware.py", line 44, in process_request
    defer.returnValue((yield download_func(request=request, spid

In [8]:
import pandas as pd

event_dfjson = pd.read_json('eventresult.json')
event_dfjson

,date,location,name,organizer-link,ticket-price
0,"Fri, Aug 23, 7:00pm + 2 more events","1270 Center Point Pkwy, Birmingham, AL",Men and Women Conference,https://www.eventbrite.com/e/men-and-women-con...,None
1,"Fri, Nov 22, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Family Law Conference for Alabama Practitioner...,https://www.eventbrite.com/e/family-law-confer...,Starts at $487.00
2,"Fri, Oct 25, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Alabama Business Law Conference (blr) S,https://www.eventbrite.com/e/alabama-business-...,Starts at $487.00
3,"Fri, Nov 1, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Alabama Real Estate Law Conference (blr) S,https://www.eventbrite.com/e/alabama-real-esta...,Starts at $487.00
4,"Fri, Oct 4, 8:30am","Holiday Inn Birmingham - Hoover, Hoover, AL",Worker's Comp Conference for Alabama Attorneys...,https://www.eventbrite.com/e/workers-comp-conf...,Starts at $487.00
5,"Wed, Nov 6, 8:00am","Homewood Church of Christ, Birmingham, AL",6th Unlocking the Mystery of Develomental Diso...,https://www.eventbrite.com/e/6th-unlocking-the...,Starts at $70.00
6,"Fri, Sep 20, 7:00pm + 2 more events","Mind Of Christ International, Alabaster, AL",Speak Lord Prophetic MEGA Conference 2019,https://www.eventbrite.com/e/speak-lord-prophe...,Starts at $30.00
7,"Thu, Aug 8, 2:00pm","Regions Field, Birmingham, AL",Magic City Money Matters Financial Conference,https://www.eventbrite.com/e/magic-city-money-...,Free
8,"Thu, Sep 12, 8:30am","Holiday Inn Birmingham - Hoover, Hoover, AL",Tort Law Conference for Alabama Attorneys (blr),https://www.eventbrite.com/e/tort-law-conferen...,Starts at $697.00
9,"Fri, Oct 11, 9:00am",Birmingham-Jefferson Civic Complex - East Exhi...,2019 CCEF Ministry Expo,https://www.eventbrite.com/e/2019-ccef-ministr...,Starts at $175.00


In [18]:
event_dfjson['ticket-price'].fillna("", inplace=True)

event_dfjson['ticket-price'] = event_dfjson['ticket-price'].map(lambda x: x.lstrip('Starts at '))
    
event_dfjson

,date,location,name,organizer-link,ticket-price
0,"Fri, Aug 23, 7:00pm + 2 more events","1270 Center Point Pkwy, Birmingham, AL",Men and Women Conference,https://www.eventbrite.com/e/men-and-women-con...,
1,"Fri, Nov 22, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Family Law Conference for Alabama Practitioner...,https://www.eventbrite.com/e/family-law-confer...,$487.00
2,"Fri, Oct 25, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Alabama Business Law Conference (blr) S,https://www.eventbrite.com/e/alabama-business-...,$487.00
3,"Fri, Nov 1, 8:00am","Holiday Inn Birmingham - Hoover, Hoover, AL",Alabama Real Estate Law Conference (blr) S,https://www.eventbrite.com/e/alabama-real-esta...,$487.00
4,"Fri, Oct 4, 8:30am","Holiday Inn Birmingham - Hoover, Hoover, AL",Worker's Comp Conference for Alabama Attorneys...,https://www.eventbrite.com/e/workers-comp-conf...,$487.00
5,"Wed, Nov 6, 8:00am","Homewood Church of Christ, Birmingham, AL",6th Unlocking the Mystery of Develomental Diso...,https://www.eventbrite.com/e/6th-unlocking-the...,$70.00
6,"Fri, Sep 20, 7:00pm + 2 more events","Mind Of Christ International, Alabaster, AL",Speak Lord Prophetic MEGA Conference 2019,https://www.eventbrite.com/e/speak-lord-prophe...,$30.00
7,"Thu, Aug 8, 2:00pm","Regions Field, Birmingham, AL",Magic City Money Matters Financial Conference,https://www.eventbrite.com/e/magic-city-money-...,Free
8,"Thu, Sep 12, 8:30am","Holiday Inn Birmingham - Hoover, Hoover, AL",Tort Law Conference for Alabama Attorneys (blr),https://www.eventbrite.com/e/tort-law-conferen...,$697.00
9,"Fri, Oct 11, 9:00am",Birmingham-Jefferson Civic Complex - East Exhi...,2019 CCEF Ministry Expo,https://www.eventbrite.com/e/2019-ccef-ministr...,$175.00
